In [1]:
from libraries import * 
from getData import * 


In [2]:
    Augmentation_settings = {
        "Cross Validation Type": "kTkV",
        "model" : "SVR",
        "metric" : "all-metrics", # not required anymore

        "Augment" : False, # not implemented yet
        # "transform" : True, # need to implement this now. 

        "verbose" : 0,    # dont change this. Taken care of 
        "top_k" : 150,

        "repeat features" : False,
        "same split" :  False, #implement false condition
        
        "stratified" : False,
        "data" : "RS",
        
        "feature reduction" : "pearason",
        "features reduced per step" : 1
        # "order" : args.order if 'stratified' in args else None, #implement false condition
    }

In [4]:
GET = getData(Augmentation_settings)
features, data, outputs = GET.data()

num_features = 150 

outputs = outputs.reshape(outputs.shape[0], 1)
data = np.hstack((data, outputs))
data = pd.DataFrame(data)

data.head()


In [7]:
# data.columns = list(features) + ["outputs"]
corrs = data.corr().abs()
importances = corrs.values[-1][:-1]

top_columns = np.argpartition(importances, -num_features)[-num_features:]
corrs


,0,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1.000000,0.231838,0.105004,0.206959,0.267749,0.631039,0.194382,0.480949,0.642868,0.043913,...,0.142265,0.118421,0.036892,0.009484,0.172658,0.191546,0.044175,0.031991,0.075524,0.011537
1,0.231838,1.000000,0.290545,0.025862,0.222755,0.163703,0.122384,0.238191,0.320355,0.171362,...,0.164165,0.108097,0.010622,0.001557,0.024834,0.133421,0.157720,0.271320,0.135513,0.273330
2,0.105004,0.290545,1.000000,0.684147,0.515099,0.118176,0.234398,0.063864,0.027732,0.428390,...,0.212825,0.147968,0.065598,0.242674,0.019615,0.260638,0.141741,0.012120,0.005646,0.033094
3,0.206959,0.025862,0.684147,1.000000,0.423187,0.321434,0.221016,0.326620,0.243296,0.169183,...,0.205537,0.074396,0.022891,0.175307,0.075826,0.006107,0.045449,0.178734,0.058090,0.048156
4,0.267749,0.222755,0.515099,0.423187,1.000000,0.225265,0.024054,0.221619,0.124094,0.067933,...,0.049686,0.218340,0.337795,0.068941,0.014292,0.143202,0.130254,0.075702,0.237466,0.110672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,0.191546,0.133421,0.260638,0.006107,0.143202,0.227571,0.025672,0.209403,0.095522,0.235482,...,0.178394,0.222179,0.336911,0.308112,0.126345,1.000000,0.198749,0.242173,0.422373,0.182413
1125,0.044175,0.157720,0.141741,0.045449,0.130254,0.261334,0.049302,0.031380,0.216983,0.025018,...,0.014361,0.233683,0.066077,0.263634,0.142017,0.198749,1.000000,0.310756,0.574969,0.171765
1126,0.031991,0.271320,0.012120,0.178734,0.075702,0.072916,0.134682,0.118834,0.142704,0.248029,...,0.254972,0.113076,0.009578,0.274958,0.018482,0.242173,0.310756,1.000000,0.100227,0.180631
1127,0.075524,0.135513,0.005646,0.058090,0.237466,0.206501,0.084420,0.196309,0.141085,0.230036,...,0.083791,0.452231,0.188752,0.270047,0.192889,0.422373,0.574969,0.100227,1.000000,0.134994


In [63]:
np.argmax(corrs.values[-1][:-1])

506

In [74]:
import heapq

In [76]:
test_top_cols = heapq.nlargest(150, range(len(importances)), importances.take)

In [81]:
test_top_cols.sort()
top_columns.sort()
# test_top_cols

In [5]:
pd.DataFrame(data).iloc[:,top_columns]

## Checking for early-fusion approach

In [9]:
df = pd.read_excel("../data/compiled_dataset_RSbivariate_without_controls_v7.xlsx", header = [0,1])

In [38]:
outputs = df[("behavioral", "wab_aq_bd")].values
outputs = outputs.reshape(len(outputs), 1)

data = {}

data["AQ"] = df[("behavioral", "wab_aq_bd")].values[:, np.newaxis]
data["DM"] = df["demographic_info"].values 


featuresFAL = ["fa_avg_ccmaj", "fa_avg_ccmin", "fa_avg_lifof", "fa_avg_lilf", "fa_avg_lslf", "fa_avg_lunc", "fa_avg_larc"]
featuresFAR = ["fa_avg_rifof", "fa_avg_rilf", "fa_avg_rslf", "fa_avg_runc", "fa_avg_rarc"]
FA_L = df["average_FA_values"][featuresFAL].fillna(0).values
FA_R = df["average_FA_values"][featuresFAR].fillna(df["average_FA_values"][featuresFAR].mean()).values
data["FA"] = np.hstack((FA_L, FA_R))

data["PS_G"] = df["percent_spared_in_gray_matter"].values
data["PS_W"] = df["percent_spared_in_white_matter"].values

data["RS"] = df["restingstate_bivariate_correlations"].values



In [4]:
import itertools

all_features = ["AQ", "DM", "FA", "PS_G", "PS_W", "RS", "LS", "BH"]
all_combinations = []
for i in range(1,len(all_features)+1):
    temp = list( map(list, list(itertools.combinations(all_features, i))))
    all_combinations += temp 

# for i in range(len(all_combinations)):
#     all_combinations[i] = "-".join(all_combinations[i])
    
# len(all_combinations)
# all_combinations.append("stan_optimal")

all_combinations

[['AQ'],
 ['DM'],
 ['FA'],
 ['PS_G'],
 ['PS_W'],
 ['RS'],
 ['LS'],
 ['BH'],
 ['AQ', 'DM'],
 ['AQ', 'FA'],
 ['AQ', 'PS_G'],
 ['AQ', 'PS_W'],
 ['AQ', 'RS'],
 ['AQ', 'LS'],
 ['AQ', 'BH'],
 ['DM', 'FA'],
 ['DM', 'PS_G'],
 ['DM', 'PS_W'],
 ['DM', 'RS'],
 ['DM', 'LS'],
 ['DM', 'BH'],
 ['FA', 'PS_G'],
 ['FA', 'PS_W'],
 ['FA', 'RS'],
 ['FA', 'LS'],
 ['FA', 'BH'],
 ['PS_G', 'PS_W'],
 ['PS_G', 'RS'],
 ['PS_G', 'LS'],
 ['PS_G', 'BH'],
 ['PS_W', 'RS'],
 ['PS_W', 'LS'],
 ['PS_W', 'BH'],
 ['RS', 'LS'],
 ['RS', 'BH'],
 ['LS', 'BH'],
 ['AQ', 'DM', 'FA'],
 ['AQ', 'DM', 'PS_G'],
 ['AQ', 'DM', 'PS_W'],
 ['AQ', 'DM', 'RS'],
 ['AQ', 'DM', 'LS'],
 ['AQ', 'DM', 'BH'],
 ['AQ', 'FA', 'PS_G'],
 ['AQ', 'FA', 'PS_W'],
 ['AQ', 'FA', 'RS'],
 ['AQ', 'FA', 'LS'],
 ['AQ', 'FA', 'BH'],
 ['AQ', 'PS_G', 'PS_W'],
 ['AQ', 'PS_G', 'RS'],
 ['AQ', 'PS_G', 'LS'],
 ['AQ', 'PS_G', 'BH'],
 ['AQ', 'PS_W', 'RS'],
 ['AQ', 'PS_W', 'LS'],
 ['AQ', 'PS_W', 'BH'],
 ['AQ', 'RS', 'LS'],
 ['AQ', 'RS', 'BH'],
 ['AQ', 'LS', 'BH'],
 ['DM', 'FA

In [81]:
from libraries import *

df = pd.read_excel("../data/compiled_dataset_RSbivariate_without_controls_v7.xlsx", header = [0,1])
outputs = df[("behavioral", "wab_aq_bd")].values
outputs = outputs.reshape(len(outputs),1)

data = {}

# AQ - Aphasia Quotient
data["AQ"] = pd.DataFrame(df[('behavioral', "wab_aq_bd")].values, columns = ["wab_aq_bd"]) 

# DM - Demographics
data["DM"] = df["demographic_info"]

# FA - Fractional Anisotropy
featuresFAL = ["fa_avg_ccmaj", "fa_avg_ccmin", "fa_avg_lifof", "fa_avg_lilf", "fa_avg_lslf", "fa_avg_lunc", "fa_avg_larc"]
featuresFAR = ["fa_avg_rifof", "fa_avg_rilf", "fa_avg_rslf", "fa_avg_runc", "fa_avg_rarc"]
FA_L = df["average_FA_values"][featuresFAL].fillna(0).values
FA_R = df["average_FA_values"][featuresFAR].fillna(df["average_FA_values"][featuresFAR].mean()).values
data["FA"] = pd.DataFrame(np.hstack((FA_L, FA_R)), columns = featuresFAL+featuresFAR)

# PS_G - percent grey matter per region  
data["PS_G"] = df["percent_spared_in_gray_matter"]

# PS_W - percent white matter per region  
data["PS_W"] = df["percent_spared_in_white_matter"]

# RS - Resting state data
data["RS"] = df["restingstate_bivariate_correlations"]

# LS - Lesion size (Total)   ; Need to add lesion size per region
data["LS"] = pd.DataFrame(df[("lesion_size", "lesion_size_ls")].values, columns = ["lesion_size_ls"])




In [96]:
def getData(feature_combo):
    features = feature_combo.split("-")
    
    correlations = []
    for i,dataset in enumerate(features):
        if i==0:
            correlations = data[dataset].values
            featureList = list(data[dataset].columns)
            
        else:
            correlations = np.hstack((correlations, data[dataset].values))
            featureList += list(data[dataset].columns)
    
    return correlations, featureList

# print(all_combinations[10])

all_features = ["AQ", "DM", "FA", "PS_G", "PS_W", "RS", "LS"]
corrdata, datafeatures = getData("AQ-DM-FA-PS_G-PS_W-RS-LS")
corrdata.shape


(55, 747)

In [101]:
data["DM"].shape

(55, 3)

In [11]:
x = "DM-RS-stan_optimal".split("_")[0]
x = x.split("-")
"_".join(x)


'DM_RS_stan'

In [22]:
import re
import pandas as pd 
import numpy as np 

x = pd.DataFrame(np.ones([5,5]), columns=["A", "B", "C", "D", "E"])
y = np.ones([5,1])
x = np.hstack((x, y))
x.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [28]:
data = pd.read_excel("../data/compiled_dataset_RSbivariate_without_controls_v7.xlsx", header = [0,1])
data["demographic_info"].shape
data[("behavioral", "difference_post_pre_bd")]

0     0.546296
1     0.000000
2     0.305556
3     0.500000
4    -0.064815
5     0.335648
6     0.194444
7     0.277778
8     0.462963
9     0.370370
10    0.194444
11    0.074074
12    0.522487
13    0.500000
14    0.157407
15    0.074074
16    0.481481
17    0.379630
18    0.166667
19    0.416667
20    0.212963
21    0.101852
22    0.037037
23    0.129630
24    0.055556
25    0.138889
26    0.287037
27    0.407407
28    0.092593
29    0.240741
30    0.240000
31    0.540000
32    0.300000
33    0.440000
34    0.160000
35    0.299851
36    0.090000
37    0.210000
38    0.388540
39    0.303958
40    0.440000
41    0.482366
42    0.193497
43    0.435933
44    0.264000
45    0.425919
46    0.450000
47    0.400000
48    0.550000
49    0.450000
50    0.650000
51    0.600000
52    0.350000
53    0.250000
54    0.000000
Name: (behavioral, difference_post_pre_bd), dtype: float64